In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import os

import tensorflow as tf
#import tensorflow_addons as tfa
from tensorflow import keras
import numpy as np



In [2]:
# Copyright 2019 The TensorFlow Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Orginal implementation from keras_contrib/layer/normalization
# =============================================================================
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import logging
import tensorflow as tf



class GroupNormalization(tf.keras.layers.Layer):
    """Group normalization layer.
    Group Normalization divides the channels into groups and computes
    within each group the mean and variance for normalization.
    Empirically, its accuracy is more stable than batch norm in a wide
    range of small batch sizes, if learning rate is adjusted linearly
    with batch sizes.
    Relation to Layer Normalization:
    If the number of groups is set to 1, then this operation becomes identical
    to Layer Normalization.
    Relation to Instance Normalization:
    If the number of groups is set to the
    input dimension (number of groups is equal
    to number of channels), then this operation becomes
    identical to Instance Normalization.
    Arguments
        groups: Integer, the number of groups for Group Normalization.
            Can be in the range [1, N] where N is the input dimension.
            The input dimension must be divisible by the number of groups.
        axis: Integer, the axis that should be normalized.
        epsilon: Small float added to variance to avoid dividing by zero.
        center: If True, add offset of `beta` to normalized tensor.
            If False, `beta` is ignored.
        scale: If True, multiply by `gamma`.
            If False, `gamma` is not used.
        beta_initializer: Initializer for the beta weight.
        gamma_initializer: Initializer for the gamma weight.
        beta_regularizer: Optional regularizer for the beta weight.
        gamma_regularizer: Optional regularizer for the gamma weight.
        beta_constraint: Optional constraint for the beta weight.
        gamma_constraint: Optional constraint for the gamma weight.
    Input shape
        Arbitrary. Use the keyword argument `input_shape`
        (tuple of integers, does not include the samples axis)
        when using this layer as the first layer in a model.
    Output shape
        Same shape as input.
    References
        - [Group Normalization](https://arxiv.org/abs/1803.08494)
    """

    def __init__(self,
                 groups=2,
                 axis=-1,
                 epsilon=1e-3,
                 center=True,
                 scale=True,
                 beta_initializer='zeros',
                 gamma_initializer='ones',
                 beta_regularizer=None,
                 gamma_regularizer=None,
                 beta_constraint=None,
                 gamma_constraint=None,
                 **kwargs):
        super(GroupNormalization, self).__init__(**kwargs)
        self.supports_masking = True
        self.groups = groups
        self.axis = axis
        self.epsilon = epsilon
        self.center = center
        self.scale = scale
        self.beta_initializer = tf.keras.initializers.get(beta_initializer)
        self.gamma_initializer = tf.keras.initializers.get(gamma_initializer)
        self.beta_regularizer = tf.keras.regularizers.get(beta_regularizer)
        self.gamma_regularizer = tf.keras.regularizers.get(gamma_regularizer)
        self.beta_constraint = tf.keras.constraints.get(beta_constraint)
        self.gamma_constraint = tf.keras.constraints.get(gamma_constraint)
        self._check_axis()

    def build(self, input_shape):

        self._check_if_input_shape_is_none(input_shape)
        self._set_number_of_groups_for_instance_norm(input_shape)
        self._check_size_of_dimensions(input_shape)
        self._create_input_spec(input_shape)

        self._add_gamma_weight(input_shape)
        self._add_beta_weight(input_shape)
        self.built = True
        super(GroupNormalization, self).build(input_shape)

    def call(self, inputs):

        input_shape = tf.keras.backend.int_shape(inputs)
        tensor_input_shape = tf.shape(inputs)

        reshaped_inputs, group_shape = self._reshape_into_groups(
            inputs, input_shape, tensor_input_shape)

        normalized_inputs = self._apply_normalization(reshaped_inputs,
                                                      input_shape)

        outputs = tf.reshape(normalized_inputs, tensor_input_shape)

        return outputs

    def get_config(self):
        config = {
            'groups':
            self.groups,
            'axis':
            self.axis,
            'epsilon':
            self.epsilon,
            'center':
            self.center,
            'scale':
            self.scale,
            'beta_initializer':
            tf.keras.initializers.serialize(self.beta_initializer),
            'gamma_initializer':
            tf.keras.initializers.serialize(self.gamma_initializer),
            'beta_regularizer':
            tf.keras.regularizers.serialize(self.beta_regularizer),
            'gamma_regularizer':
            tf.keras.regularizers.serialize(self.gamma_regularizer),
            'beta_constraint':
            tf.keras.constraints.serialize(self.beta_constraint),
            'gamma_constraint':
            tf.keras.constraints.serialize(self.gamma_constraint)
        }
        base_config = super(GroupNormalization, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

    def compute_output_shape(self, input_shape):
        return input_shape

    def _reshape_into_groups(self, inputs, input_shape, tensor_input_shape):

        group_shape = [tensor_input_shape[i] for i in range(len(input_shape))]
        group_shape[self.axis] = input_shape[self.axis] // self.groups
        group_shape.insert(self.axis, self.groups)
        group_shape = tf.stack(group_shape)
        reshaped_inputs = tf.reshape(inputs, group_shape)
        return reshaped_inputs, group_shape

    def _apply_normalization(self, reshaped_inputs, input_shape):

        group_shape = tf.keras.backend.int_shape(reshaped_inputs)
        group_reduction_axes = list(range(1, len(group_shape)))
        axis = -2 if self.axis == -1 else self.axis - 1
        group_reduction_axes.pop(axis)

        mean, variance = tf.nn.moments(
            reshaped_inputs, group_reduction_axes, keepdims=True)

        gamma, beta = self._get_reshaped_weights(input_shape)
        normalized_inputs = tf.nn.batch_normalization(
            reshaped_inputs,
            mean=mean,
            variance=variance,
            scale=gamma,
            offset=beta,
            variance_epsilon=self.epsilon)
        return normalized_inputs

    def _get_reshaped_weights(self, input_shape):
        broadcast_shape = self._create_broadcast_shape(input_shape)
        gamma = None
        beta = None
        if self.scale:
            gamma = tf.reshape(self.gamma, broadcast_shape)

        if self.center:
            beta = tf.reshape(self.beta, broadcast_shape)
        return gamma, beta

    def _check_if_input_shape_is_none(self, input_shape):
        dim = input_shape[self.axis]
        if dim is None:
            raise ValueError('Axis ' + str(self.axis) + ' of '
                             'input tensor should have a defined dimension '
                             'but the layer received an input with shape ' +
                             str(input_shape) + '.')

    def _set_number_of_groups_for_instance_norm(self, input_shape):
        dim = input_shape[self.axis]

        if self.groups == -1:
            self.groups = dim

    def _check_size_of_dimensions(self, input_shape):

        dim = input_shape[self.axis]
        if dim < self.groups:
            raise ValueError(
                'Number of groups (' + str(self.groups) + ') cannot be '
                'more than the number of channels (' + str(dim) + ').')

        if dim % self.groups != 0:
            raise ValueError(
                'Number of groups (' + str(self.groups) + ') must be a '
                'multiple of the number of channels (' + str(dim) + ').')

    def _check_axis(self):

        if self.axis == 0:
            raise ValueError(
                "You are trying to normalize your batch axis. Do you want to "
                "use tf.layer.batch_normalization instead")

    def _create_input_spec(self, input_shape):

        dim = input_shape[self.axis]
        self.input_spec = tf.keras.layers.InputSpec(
            ndim=len(input_shape), axes={self.axis: dim})

    def _add_gamma_weight(self, input_shape):

        dim = input_shape[self.axis]
        shape = (dim,)

        if self.scale:
            self.gamma = self.add_weight(
                shape=shape,
                name='gamma',
                initializer=self.gamma_initializer,
                regularizer=self.gamma_regularizer,
                constraint=self.gamma_constraint)
        else:
            self.gamma = None

    def _add_beta_weight(self, input_shape):

        dim = input_shape[self.axis]
        shape = (dim,)

        if self.center:
            self.beta = self.add_weight(
                shape=shape,
                name='beta',
                initializer=self.beta_initializer,
                regularizer=self.beta_regularizer,
                constraint=self.beta_constraint)
        else:
            self.beta = None

    def _create_broadcast_shape(self, input_shape):
        broadcast_shape = [1] * len(input_shape)
        broadcast_shape[self.axis] = input_shape[self.axis] // self.groups
        broadcast_shape.insert(self.axis, self.groups)
        return broadcast_shape


class InstanceNormalization(GroupNormalization):
    """Instance normalization layer.
    Instance Normalization is an specific case of ```GroupNormalization```since
    it normalizes all features of one channel. The Groupsize is equal to the
    channel size. Empirically, its accuracy is more stable than batch norm in a
    wide range of small batch sizes, if learning rate is adjusted linearly
    with batch sizes.
    Arguments
        axis: Integer, the axis that should be normalized.
        epsilon: Small float added to variance to avoid dividing by zero.
        center: If True, add offset of `beta` to normalized tensor.
            If False, `beta` is ignored.
        scale: If True, multiply by `gamma`.
            If False, `gamma` is not used.
        beta_initializer: Initializer for the beta weight.
        gamma_initializer: Initializer for the gamma weight.
        beta_regularizer: Optional regularizer for the beta weight.
        gamma_regularizer: Optional regularizer for the gamma weight.
        beta_constraint: Optional constraint for the beta weight.
        gamma_constraint: Optional constraint for the gamma weight.
    Input shape
        Arbitrary. Use the keyword argument `input_shape`
        (tuple of integers, does not include the samples axis)
        when using this layer as the first layer in a model.
    Output shape
        Same shape as input.
    References
        - [Instance Normalization: The Missing Ingredient for Fast Stylization]
        (https://arxiv.org/abs/1607.08022)
    """

    def __init__(self, **kwargs):
        if "groups" in kwargs:
            logging.warning("The given value for groups will be overwritten.")

        kwargs["groups"] = -1
        super(InstanceNormalization, self).__init__(**kwargs)

In [3]:
def Resnet_identity_block(X,kernelsize,num_filters,dilation):
    X_shortcut = X
    #First Component
    X = keras.layers.Conv2D(filters = num_filters , kernel_size = kernelsize, padding = 'same', dilation_rate = dilation)(X)
    X = InstanceNormalization()(X)
    X = keras.layers.Activation('relu')(X)
    X = keras.layers.Dropout(rate = 0.15)(X)
    
    
    # Second component of main path
    X = keras.layers.Conv2D(filters = num_filters , kernel_size = kernelsize, padding = 'same', dilation_rate = dilation)(X)
    X = InstanceNormalization()(X)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation
    X = keras.layers.Add()([X, X_shortcut])
    X = keras.layers.Activation('relu')(X)
    
    return X


In [4]:
import tensorflow.keras.backend as K


def customLoss(yTrue,yPred):

    if yTrue.shape[0] == None:
        return 1e-6
    
    print(yPred)
    yPred= tf.clip_by_value(yPred, 1e-6, (1. - 1e-6))
    mask=K.less_equal(yTrue,2)
    
    return tf.reduce_mean(K.categorical_crossentropy(tf.one_hot(tf.cast(tf.boolean_mask(yTrue, mask),tf.int32), 3),tf.boolean_mask(yPred, mask)))

  

import tensorflow.keras.backend as K
def crossentropy_cut(y_true,y_pred):
    print(y_true.shape)
    print(y_pred.shape)
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    y_pred_f= tf.clip_by_value(y_pred_f, 1e-7, (1. - 1e-7))
    mask=K.greater_equal(y_true_f,-0.5)
    losses = -(y_true_f * K.log(y_pred_f) + (1.0 - y_true_f) * K.log(1.0 - y_pred_f))
    losses = tf.boolean_mask(losses, mask)
    masked_loss = tf.reduce_mean(losses)
    return masked_loss

In [5]:
def resnet():
    X_input = keras.layers.Input(shape=(None,None,526))
    X = keras.layers.Conv2D(filters = 64 , kernel_size = 1, padding = 'same')(X_input)
    X = InstanceNormalization()(X)
    X = keras.layers.Activation('relu')(X)
    for i in range(36):
        X=Resnet_identity_block(X,3,64,2**(i%5))
    X = keras.layers.Conv2D(filters = 3 , kernel_size = 1, padding = 'same')(X)
    X = keras.layers.Activation('relu')(X)

    X_col = tf.reduce_mean(X,axis = 1)
    X_row = tf.reduce_mean(X,axis = 2)
    X = tf.math.add(X_col,X_row)
    X = tf.math.multiply(X,0.5)

    X = keras.layers.Conv1D(filters = 3 , kernel_size = 1, padding = 'same')(X)
    X = keras.layers.Activation('softmax')(X)
    model = keras.models.Model(inputs = X_input, outputs = X)
    model.compile(optimizer=tf.keras.optimizers.Adam(), loss=customLoss)
    return model

In [6]:
def input_generator(input_feature_dir, input_label_dir,epochs):
    directory = os.fsencode(input_feature_dir)
    for j in range(epochs):
        for file in os.listdir(directory):
            
            filename = os.fsdecode(file)
            
            yield np.load(os.path.join(input_feature_dir,filename)), np.load(os.path.join(input_label_dir,filename))

def input_generator(input_feature_dir, input_label_dir,epochs):
    directory = os.fsencode(input_feature_dir)
    for j in range(epochs):
        for file in os.listdir(directory):
            
            filename = "1o13A.npy"

            
            yield np.load("/nfs/amino-home/qingyliu/dihedral_angle/conv_ML_input/1o13A_nonan.npy"), np.expand_dims(np.load(os.path.join(input_label_dir,filename)),axis = 0)

In [7]:
checkpoint_path_all_conv = "/home/qingyliu/test/training_checkpoint/test"
checkpoint_dir_all_conv = os.path.dirname(checkpoint_path_all_conv)
cp_callback_all_conv = tf.keras.callbacks.ModelCheckpoint(checkpoint_path_all_conv,
                                                 save_weights_only=True,
                                                 verbose=1,
                                                 save_freq=2        )

In [8]:
model = resnet()

In [ ]:

model.fit_generator(input_generator("/nfs/amino-home/qingyliu/dihedral_angle/temp", "/nfs/amino-home/qingyliu/dihedral_angle/chi1_label",epochs = 500),
                             steps_per_epoch = 1,
                             epochs = 500,
                             callbacks = [cp_callback_all_conv])

Epoch 1/500
tf.Tensor(
[[[0.6768406  0.04110928 0.28205013]
  [0.78910416 0.01712632 0.19376956]
  [0.6619796  0.01623361 0.3217868 ]
  [0.6575015  0.0252161  0.3172824 ]
  [0.68477446 0.01065021 0.3045753 ]
  [0.743647   0.00973902 0.24661396]
  [0.73387784 0.00989666 0.25622547]
  [0.7913956  0.0035379  0.20506646]
  [0.68678033 0.01244625 0.30077344]
  [0.6544598  0.00674373 0.33879656]
  [0.6992179  0.01180179 0.28898025]
  [0.6862218  0.01117802 0.30260023]
  [0.72964835 0.00726885 0.2630828 ]
  [0.7249161  0.02072339 0.25436047]
  [0.8121087  0.00976084 0.17813043]
  [0.770361   0.00741098 0.22222799]
  [0.90673834 0.00356801 0.08969372]
  [0.90367275 0.00896719 0.08736004]
  [0.8894678  0.01292539 0.09760676]
  [0.87953514 0.00783994 0.11262487]
  [0.8875935  0.01816051 0.09424605]
  [0.8898149  0.01055626 0.09962879]
  [0.90672636 0.00598893 0.08728474]
  [0.9084308  0.00384107 0.08772817]
  [0.9125715  0.0029947  0.08443385]
  [0.9261793  0.00467352 0.0691472 ]
  [0.9113771  0


Epoch 00002: saving model to /home/qingyliu/test/training_checkpoint/test
1/1 [==============================] - 8s 8s/step - loss: 2.7977
Epoch 3/500
tf.Tensor(
[[[5.41273177e-01 7.13760406e-02 3.87350827e-01]
  [5.59796870e-01 5.46679795e-02 3.85535151e-01]
  [5.80742836e-01 1.15831792e-01 3.03425372e-01]
  [6.36162043e-01 9.57356915e-02 2.68102258e-01]
  [5.60142934e-01 1.93246827e-01 2.46610269e-01]
  [6.27990961e-01 1.58780754e-01 2.13228270e-01]
  [6.47249639e-01 1.58642888e-01 1.94107398e-01]
  [6.49678469e-01 1.57632917e-01 1.92688704e-01]
  [4.86470014e-01 3.84083509e-01 1.29446462e-01]
  [4.10048902e-01 5.07690012e-01 8.22610334e-02]
  [3.74570012e-01 5.47128737e-01 7.83012435e-02]
  [2.93810099e-01 6.44360662e-01 6.18292205e-02]
  [4.74223197e-01 4.31872636e-01 9.39041823e-02]
  [3.06761712e-01 6.45734847e-01 4.75034043e-02]
  [3.28161031e-01 6.24771714e-01 4.70672436e-02]
  [2.66091973e-01 7.03531265e-01 3.03767063e-02]
  [5.39238751e-02 9.38644528e-01 7.43159419e-03]
  [5


Epoch 00004: saving model to /home/qingyliu/test/training_checkpoint/test
1/1 [==============================] - 8s 8s/step - loss: 3.2852
Epoch 5/500
tf.Tensor(
[[[0.43948415 0.2638154  0.29670048]
  [0.4366835  0.28238386 0.28093266]
  [0.43468124 0.30702204 0.25829673]
  [0.4759361  0.20806892 0.315995  ]
  [0.49751374 0.13047506 0.37201115]
  [0.5188056  0.15326113 0.32793322]
  [0.49782938 0.1644263  0.33774427]
  [0.53070855 0.10207319 0.36721826]
  [0.45503014 0.32073298 0.22423688]
  [0.49614546 0.24373664 0.2601179 ]
  [0.49832153 0.24548753 0.25619093]
  [0.4746618  0.25653845 0.26879972]
  [0.5323099  0.23260978 0.23508027]
  [0.49300244 0.23721568 0.2697818 ]
  [0.52898633 0.21425955 0.2567541 ]
  [0.5347276  0.20043394 0.26483852]
  [0.3749112  0.4445157  0.18057312]
  [0.41946182 0.38279915 0.19773905]
  [0.39171022 0.4152949  0.19299485]
  [0.40886077 0.39159593 0.19954328]
  [0.44702604 0.33510202 0.21787192]
  [0.4396817  0.32004914 0.24026915]
  [0.46284416 0.3231845